# Spanish Translation A/B Test

## Overview
    
   * Given an A/B test results, want to know:
      * Explore the test results
      * Automate the process

In [58]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline

In [59]:
# read data
test_table = pd.read_csv('test_table.csv')
user_table = pd.read_csv('user_table.csv')

## 1. Desrciptive Data Analysis
   * 1.1 Column Structure
   * 1.2 Missing Value
   * 1.3 Outlier

In [60]:
# Starting from test table
test_table.head()

,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test
0,315281,2015-12-03,Direct,Web,ES,NaN,IE,1,0
1,497851,2015-12-04,Ads,Web,ES,Google,IE,0,1
2,848402,2015-12-04,Ads,Web,ES,Facebook,Chrome,0,0
3,290051,2015-12-03,Ads,Mobile,Other,Facebook,Android_App,0,1
4,548435,2015-11-30,Ads,Web,ES,Google,FireFox,0,1


In [61]:
# table structure
display(test_table.describe(include=np.number))
display(test_table.describe(include=np.object))
# check on user ID, primary key cannot has duplicates
len(test_table['user_id'].unique())

,user_id,conversion,test
count,453321.000000,453321.000000,453321.000000
mean,499937.514728,0.049579,0.476446
std,288665.193436,0.217073,0.499445
min,1.000000,0.000000,0.000000
25%,249816.000000,0.000000,0.000000
50%,500019.000000,0.000000,0.000000
75%,749522.000000,0.000000,1.000000
max,1000000.000000,1.000000,1.000000


,date,source,device,browser_language,ads_channel,browser
count,453321,453321,453321,453321,181877,453321
unique,5,3,2,3,5,7
top,2015-12-04,Ads,Web,ES,Facebook,Android_App
freq,141163,181877,251565,377547,68425,155135


453321

In [62]:
# For catogorical columns
display(test_table['date'].value_counts())
display(test_table['source'].value_counts())
display(test_table['device'].value_counts())
display(test_table['browser_language'].value_counts())
display(test_table['ads_channel'].value_counts())
print(test_table['browser'].value_counts())

2015-12-04    141163
2015-12-03     99493
2015-11-30     71025
2015-12-01     70991
2015-12-02     70649
Name: date, dtype: int64

Ads       181877
SEO       180610
Direct     90834
Name: source, dtype: int64

Web       251565
Mobile    201756
Name: device, dtype: int64

ES       377547
EN        63137
Other     12637
Name: browser_language, dtype: int64

Facebook    68425
Google      68180
Yahoo       27435
Bing        13689
Other        4148
Name: ads_channel, dtype: int64

Android_App    155135
Chrome         101929
IE              61715
Iphone_App      46621
Safari          41065
FireFox         40766
Opera            6090
Name: browser, dtype: int64


In [63]:
# for user table
display(user_table.describe(include=np.number))
display(user_table.describe(include=np.object))
# check on user ID, primary key cannot has duplicates
len(user_table['user_id'].unique())

,user_id,age
count,452867.000000,452867.000000
mean,499944.805166,27.130740
std,288676.264784,6.776678
min,1.000000,18.000000
25%,249819.000000,22.000000
50%,500019.000000,26.000000
75%,749543.000000,31.000000
max,1000000.000000,70.000000


,sex,country
count,452867,452867
unique,2,17
top,M,Mexico
freq,264485,128484


452867

In [64]:
display(user_table['sex'].value_counts())
display(user_table['country'].value_counts())

M    264485
F    188382
Name: sex, dtype: int64

Mexico         128484
Colombia        54060
Spain           51782
Argentina       46733
Peru            33666
Venezuela       32054
Chile           19737
Ecuador         15895
Guatemala       15125
Bolivia         11124
Honduras         8568
El Salvador      8175
Paraguay         7347
Nicaragua        6723
Costa Rica       5309
Uruguay          4134
Panama           3951
Name: country, dtype: int64

In [65]:
# notice the user_id in user table has 452867 unique value, in test_table it has more, 
# wonder if some records in test table cannot find a corresponding user
unq_id = set(user_table['user_id'].unique())
flag = test_table['user_id'].apply(lambda x: True if x not in unq_id else False)
print('number of user_id that does not exist in the user table: {}'.format(sum(flag)))

number of user_id that does not exist in the user table: 454


In [66]:
# examples of those records
test_table[flag].head()

,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test
819,657841,2015-12-02,Ads,Mobile,ES,Bing,Android_App,0,0
1696,859340,2015-12-04,Direct,Mobile,EN,NaN,Android_App,0,0
1934,716487,2015-12-04,SEO,Mobile,ES,NaN,Android_App,0,0
2409,87209,2015-12-01,Direct,Web,ES,NaN,Chrome,0,0
2721,771642,2015-11-30,SEO,Mobile,ES,NaN,Android_App,0,0


In [67]:
# cant find noticeable pattern, since we don't know what country they are from, we could discard those records
# redefine test table
test_table = test_table[~flag] 

In [68]:
# now let's check null/missing data
# start from test table
display(test_table.isnull().describe()) # only ads channel has null/na value, which makes sense since not all users are from ADS
display(test_table[test_table.isnull().any(axis = 1)].head()) # some examples

,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test
count,452867,452867,452867,452867,452867,452867,452867,452867,452867
unique,1,1,1,1,1,2,1,1,1
top,False,False,False,False,False,True,False,False,False
freq,452867,452867,452867,452867,452867,271174,452867,452867,452867


,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test
0,315281,2015-12-03,Direct,Web,ES,NaN,IE,1,0
5,540675,2015-12-03,Direct,Mobile,ES,NaN,Android_App,0,1
6,863394,2015-12-04,SEO,Mobile,Other,NaN,Android_App,0,0
7,527287,2015-12-03,Direct,Web,EN,NaN,Chrome,0,0
8,261625,2015-12-04,Direct,Mobile,ES,NaN,Android_App,0,1


In [69]:
# then for user table
display(user_table.isnull().describe()) # no missing values
display(user_table[user_table.isnull().any(axis = 1)].head()) # some examples

,user_id,sex,age,country
count,452867,452867,452867,452867
unique,1,1,1,1
top,False,False,False,False
freq,452867,452867,452867,452867


,user_id,sex,age,country


## Summary for Desrciptive Data Analysis
   * 1.1 Column Structure
       * 452867 distinct user_id
       * age from 18 - 70
       * ...
   * 1.2 Missing Value
       * Removed 454 records from test because they are not in the user table
   * 1.3 Outlier
       * Nothing noticable

## 2. A/B test
   * Goal is to figure out which version of spanish translation is better
   * Sanity Check
       * if invariant variable are truely invariant 
       * sign test
   * Significance check

In [70]:
# first of all, join two tables
df = pd.merge(left = test_table, right = user_table, how = 'left', on = 'user_id', suffixes=(False, '_user'))

In [71]:
df.head()

,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test,sex,age,country
0,315281,2015-12-03,Direct,Web,ES,NaN,IE,1,0,M,32,Spain
1,497851,2015-12-04,Ads,Web,ES,Google,IE,0,1,M,21,Mexico
2,848402,2015-12-04,Ads,Web,ES,Facebook,Chrome,0,0,M,34,Spain
3,290051,2015-12-03,Ads,Mobile,Other,Facebook,Android_App,0,1,F,22,Mexico
4,548435,2015-11-30,Ads,Web,ES,Google,FireFox,0,1,M,19,Mexico


In [72]:
# then exclude Spain from the data, since they are all control instead of exp
spain_df = df[df['country'] == 'Spain']
df = df[df['country'] != 'Spain']

In [73]:
# I would like to know if test is only on browser which has language set up as ES
# if ture, then we will need to exclude all other/EN language from the dataset as well
df.groupby(['browser_language', 'test']).agg({'test':np.size, 'conversion': np.sum})
# indeed there is test cases set up for browser using other languages, thoses shouldnt have a impact on our test
# exclude them as well
other_language_df = df[df['browser_language'] != 'ES']
df = df[df['browser_language'] == 'ES']

In [74]:
# now let's take a look at the data
tmp = df.groupby(['date', 'test']).agg({'test':np.size, 'conversion': np.sum})
tmp['conversion_rate'] = tmp['conversion']/tmp['test']
tmp.rename(columns={'test':'sample'}, inplace=True)
tmp

sample  conversion  conversion_rate
date       test                                     
2015-11-30 0      24236        1244         0.051329
           1      28213        1264         0.044802
2015-12-01 0      24178        1147         0.047440
           1      28223        1164         0.041243
2015-12-02 0      24163        1180         0.048835
           1      27891        1234         0.044244
2015-12-03 0      33898        1674         0.049383
           1      39401        1726         0.043806
2015-12-04 0      47941        2263         0.047204
           1      55870        2428         0.043458

In [75]:
# Test group always has more samples than control group on everyday, which is not normal
# lets do a validation on the data distribution, see if it is really 5050 split between test and experiment
def check_invariant_binomial(x_n, y_n, p = .5, alpha = .05):
    '''check if the sample size are equal on control and experiment
    
    :param x_n: is the # of event happened in x group
    :param y_n: is the # of same event happened in y group
    :param p: is the probability of event happening in group x, default .5
    :param alpha: is the signifcant level, default .05
    '''
    total_event = x_n + y_n
    SE = (p*(1-p)/total_event)**.5
    diff = x_n/total_event - p
    
    m = abs(scipy.stats.norm.ppf(alpha/2))*SE
    
    
    if diff>=-m and diff<= m:
        print('Invariant')
        print('diff: {0:.4f}'.format(diff)) 
        print('interval: [{0:.4f}, {1:.4f}]'.format(-m, m))
        return True
    else: 
        print('Not invariant')
        print('diff: {0:.4f}'.format(diff)) 
        print('interval: [{0:.4f}, {1:.4f}]'.format(-m, m))
        return False

In [76]:
x_n = df[df['test'] == 1].size
y_n = df[df['test'] == 0].size
check_invariant_binomial(x_n, y_n, p = .5, alpha = .05)

Not invariant
diff: 0.0377
interval: [-0.0005, 0.0005]


False

In [77]:
# Take a closer look at each country, see if this behavior is related to countries
tmp = df.groupby(['country', 'test']).agg({'test':np.size, 'conversion': np.sum})
tmp['conversion_rate'] = tmp['conversion']/tmp['test']
tmp.rename(columns={'test':'sample'}, inplace=True)
tmp

sample  conversion  conversion_rate
country     test                                     
Argentina   0       7793         120         0.015398
            1      31204         439         0.014069
Bolivia     0       4600         228         0.049565
            1       4654         230         0.049420
Chile       0       8262         399         0.048293
            1       8171         406         0.049688
Colombia    0      22502        1190         0.052884
            1      22348        1140         0.051011
Costa Rica  0       2196         113         0.051457
            1       2182         116         0.053162
Ecuador     0       6746         340         0.050400
            1       6516         318         0.048803
El Salvador 0       3420         187         0.054678
            1       3355         164         0.048882
Guatemala   0       6373         318         0.049898
            1       6237         320         0.051307
Honduras    0       3636         184         0.050605
            1       3531         173         0.048995
Mexico      0      53550        2676         0.049972
            1      53536        2727         0.050938
Nicaragua   0       2869         155         0.054026
            1       2767         151         0.054572
Panama      0       1628          78         0.047912
            1       1662          86         0.051745
Paraguay    0       3007         142         0.047223
            1       3082         145         0.047047
Peru        0      13990         691         0.049392
            1      14010         707         0.050464
Uruguay     0        356           3         0.008427
            1       3098          34         0.010975
Venezuela   0      13488         684         0.050712
            1      13245         660         0.049830

In [78]:
# Uruguay and Argentina have way more test group then the control group
# take a look at what happened
display(df[(df['country'] == 'Uruguay') & (df['test'] == 1)].head())
display(df[(df['country'] == 'Uruguay') & (df['test'] == 0)].head())
# Argentina
display(df[(df['country'] == 'Argentina') & (df['test'] == 1)].head())
display(df[(df['country'] == 'Argentina') & (df['test'] == 0)].head())
# Nothing significant, probably something wrong with the experiment setting in those to countries

,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test,sex,age,country
11,73335,2015-12-03,SEO,Web,ES,NaN,IE,0,1,F,29,Uruguay
61,504226,2015-12-04,Ads,Web,ES,Google,Chrome,0,1,M,48,Uruguay
200,427346,2015-12-04,SEO,Mobile,ES,NaN,Android_App,0,1,M,25,Uruguay
207,466913,2015-12-03,Ads,Web,ES,Other,Chrome,0,1,F,19,Uruguay
408,553216,2015-12-02,Ads,Web,ES,Facebook,Chrome,0,1,F,27,Uruguay


,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test,sex,age,country
2220,291594,2015-12-03,Ads,Web,ES,Other,IE,0,0,F,21,Uruguay
3891,525531,2015-12-02,Direct,Web,ES,NaN,Safari,0,0,M,18,Uruguay
7596,176297,2015-12-03,Ads,Mobile,ES,Facebook,Android_App,0,0,M,20,Uruguay
8416,467265,2015-11-30,Ads,Web,ES,Facebook,IE,0,0,M,20,Uruguay
8699,169937,2015-12-01,SEO,Mobile,ES,NaN,Android_App,0,0,M,26,Uruguay


,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test,sex,age,country
49,310570,2015-12-01,SEO,Web,ES,NaN,Safari,0,1,M,31,Argentina
57,216139,2015-12-01,Direct,Web,ES,NaN,IE,0,1,F,25,Argentina
73,812673,2015-12-03,Direct,Mobile,ES,NaN,Android_App,0,1,F,33,Argentina
75,147735,2015-11-30,SEO,Mobile,ES,NaN,Android_App,0,1,M,25,Argentina
80,159812,2015-12-01,Direct,Mobile,ES,NaN,Android_App,0,1,M,39,Argentina


,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test,sex,age,country
31,730926,2015-11-30,Ads,Web,ES,Facebook,IE,0,0,F,26,Argentina
107,82356,2015-11-30,Direct,Web,ES,NaN,Safari,0,0,M,22,Argentina
219,823663,2015-12-04,SEO,Web,ES,NaN,Safari,0,0,M,34,Argentina
248,895992,2015-12-02,Direct,Mobile,ES,NaN,Android_App,0,0,F,28,Argentina
281,446524,2015-11-30,SEO,Web,ES,NaN,Safari,0,0,M,32,Argentina


In [79]:
# let's exclude those and start AB test
df_1 = df[(df['country'] != 'Argentina') & (df['country'] != 'Uruguay')]

In [85]:
# def binomial AB test function
# return difference, confidence interval
def AB_test(ctl, exp, label, alpha):
    """Function that returns confidence interval for the probability difference of two groups
    
    :param x: pandas dataframe for AB test analysis, control group
    :param y: pandas dataframe for AB test analysis, exp group
    :param label: column names for label; column should only has 0/1 two values
    :param alpha: alpha level
    
    :return p_diff: difference between probablites of two group
    :return con_inv: confidence interval at alpha level for the difference bewteen two groups
    
    """
    x_ctl = ctl[label].sum()
    n_ctl = ctl[label].size
    p_ctl = x_ctl/n_ctl
    x_exp = exp[label].sum()
    n_exp = exp[label].size
    p_exp = x_exp/n_exp
    p_diff = p_exp - p_ctl
    p_pool = (x_ctl+x_exp)/(n_ctl+n_exp)
    pool_se = (p_pool*(1-p_pool)*(1/n_exp + 1/n_ctl))**0.5
    z = scipy.stats.norm.ppf(1-alpha/2)
    print(z)
    con_inv = [p_diff - z*pool_se, p_diff + z*pool_se]
    
    return p_diff, con_inv

In [86]:
# lets run the test with Argentina and Uruguay
ctl = df[df['test'] == 0]
exp = df[df['test'] == 1]
diff, con_inv  = AB_test(ctl = ctl, exp = exp, label = 'conversion', alpha = 0.05)
print('conversion difference between two groups: {0:.5f}'.format(diff))
print('confidence interval for the conversion difference: [{0:.5f}, {1:.5f}]'.format(con_inv[0],con_inv[1]))

1.959963984540054
conversion difference between two groups: -0.00510
confidence interval for the conversion difference: [-0.00653, -0.00368]


In [87]:
# lets run the test without Argentina and Uruguay
ctl = df_1[df_1['test'] == 0]
exp = df_1[df_1['test'] == 1]
diff, con_inv  = AB_test(ctl = ctl, exp = exp, label = 'conversion', alpha = 0.05)
print('conversion difference between two groups: {0:.5f}'.format(diff))
print('confidence interval for the conversion difference: [{0:.5f}, {1:.5f}]'.format(con_inv[0],con_inv[1]))

1.959963984540054
conversion difference between two groups: 0.00005
confidence interval for the conversion difference: [-0.00154, 0.00164]


In [88]:
# So what we find is really interesting, if we include Ar

In [82]:
# pretty much no significant difference between control/experiment group
# lets run an sign test to verify
import math
def signtest(signarray):
    '''two tail sign test for n < 30, using binomial calculation
    
    :param signarray: list of boolean that means the result of the sign test
    
    :return p: possibility that more extreme cases can happen
    '''
    
    n = len(signarray)
    t = sum(signarray)
    p = 0
    for i in range(n+1):
        if i <= min(t, n-t) or i >= max(n-t,t):
            p += math.factorial(n)/math.factorial(i)/math.factorial(n-i)*(0.5**n)

    return p

In [83]:
# prepare the sign array
tmp = df.groupby(['date', 'test']).agg({'conversion':np.sum, 'test':np.size})
tmp['conversion_rate'] = tmp['conversion']/tmp['test']
tmp.rename(columns = {'test': 'size'}, inplace = True)
tmp.reset_index(inplace = True)

sign = []
for date in tmp['date'].unique():
    exp_flag = (tmp['date'] == date) & (tmp['test'] == 1)
    ctl_flag = (tmp['date'] == date) & (tmp['test'] == 0)
    sign.append(tmp[exp_flag]['conversion_rate'].values[0] > tmp[ctl_flag]['conversion_rate'].values[0])
    
display(tmp)
print('probability that more extreme cases would happen: {0:.4f}'.format(signtest(sign)))

,date,test,conversion,size,conversion_rate
0,2015-11-30,0,1244,24236,0.051329
1,2015-11-30,1,1264,28213,0.044802
2,2015-12-01,0,1147,24178,0.047440
3,2015-12-01,1,1164,28223,0.041243
4,2015-12-02,0,1180,24163,0.048835
5,2015-12-02,1,1234,27891,0.044244
6,2015-12-03,0,1674,33898,0.049383
7,2015-12-03,1,1726,39401,0.043806
8,2015-12-04,0,2263,47941,0.047204
9,2015-12-04,1,2428,55870,0.043458


probability that more extreme cases would happen: 0.0625


In [84]:
# p value is 1, meaning no significant difference between control and experiment group